In [1]:
import numpy as np
import pickle
from learnable_crf import LearnableCrf
from lib import *
from scipy.special import expit as sigmoid

In [2]:
with open('cache/df_val_test.pickle', mode='rb') as h:
    df_val, df_test = pickle.load(h)
Y_val = df_val['label']
Y_test = df_test['label']
with open('cache/hex.pickle', mode='rb') as h:
    hex_data = pickle.load(h)
id_fh = map(lambda x: list(max(x, key=len)), hex_data['id_hierarchical_labels'])  # convert to list for addressing
state_space = hex_data['state_space']
# state_space = filter(lambda x: x[:20].any(), state_space)

In [3]:
r = 0
with open('cache/df_train.{}.pickle'.format(r), mode='rb') as h:
    df = pickle.load(h)
leaves = np.nonzero([x[0] in x[1] for x in zip(df['label'], df['pseudo_label'])])[0]
Y_train = df['label'][leaves]

In [4]:
model_Phi_train = np.load('results/caffe_independent/iter_Phi_train.{}.npy'.format(r))[:, leaves]
model_Phi_val = np.load('results/caffe_independent/iter_Phi_val.{}.npy'.format(r))
model_Phi_test = np.load('results/caffe_independent/iter_Phi_test.{}.npy'.format(r))
M = len(model_Phi_train)

In [5]:
def confusion_matrix(Y_predict, Y_truth):
    if Y_predict.dtype == bool:
        cm = np.zeros((27, 27), dtype=int)
        count = np.zeros(27, dtype=int)
        for i, y in enumerate(Y_predict):
            count[id_fh[Y_truth[i]]] += 1
            cm[id_fh[Y_truth[i]], :] += y
    else:
        cm = np.zeros((20, 27), dtype=int)
        count = np.zeros(20, dtype=int)
        for i, y in enumerate(Y_predict):
            count[Y_truth[i]] += 1
            cm[Y_truth[i], y.argmax()] += 1
    return cm.astype(float) / count[:, None]

In [ ]:
def raw_crf(Phi):
    def step(phi):
        scores = map(lambda s: phi[s].sum(), state_space)
#         return np.vstack(tuple(state_space[np.argsort(scores)[i]] for i in range(-5, 0)))
        return state_space[np.argmax(scores)]
    return np.array(map(step, Phi), dtype=bool)

In [ ]:
# [get_accuracy(model_Phi_val[i], Y_val, lim_states=False) for i in range(0, M)]
[get_accuracy(raw_crf(model_Phi_val[i]), Y_val) for i in range(0, M)]

In [ ]:
# [get_accuracy(model_Phi_test[i], Y_test, lim_states=False) for i in range(0, M)]
[get_accuracy(raw_crf(model_Phi_test[i]), Y_test) for i in range(0, M)]

In [ ]:
# np.diagonal(confusion_matrix(model_Phi_test[1], Y_test))
np.diagonal(confusion_matrix(raw_crf(model_Phi_test[1]), Y_test))

In [5]:
model_Phi_train = sigmoid(model_Phi_train)
model_Phi_val = sigmoid(model_Phi_val)
model_Phi_test = sigmoid(model_Phi_test)
lcrf = [LearnableCrf(model_Phi_train[i], Y_train) for i in range(0, M)]
[get_accuracy(lcrf[i].predict(model_Phi_val[i]), Y_val) for i in range(0, M)]

[0.39489311163895485,
 0.42695961995249404,
 0.4412114014251782,
 0.44358669833729214,
 0.44299287410926363,
 0.44358669833729214,
 0.44418052256532065,
 0.44418052256532065,
 0.44418052256532065,
 0.44418052256532065]

In [6]:
[get_accuracy(lcrf[i].predict(model_Phi_test[i]), Y_test) for i in range(0, M)]

[0.40498812351543945,
 0.4334916864608076,
 0.4513064133016627,
 0.45308788598574823,
 0.45546318289786225,
 0.45605700712589076,
 0.45605700712589076,
 0.45605700712589076,
 0.45605700712589076,
 0.45605700712589076]

In [8]:
np.diagonal(confusion_matrix(lcrf[-1].predict(model_Phi_test[-1]), Y_test))

array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.83823529,  0.        ,  0.81481481,
        0.        ,  0.        ,  0.        ,  0.        ,  0.65217391,
        0.23529412,  0.        ,  0.        ,  0.85849057,  0.        ,
        0.75486381,  0.66666667,  0.95341615,  0.93192488,  0.87484036,
        0.70967742,  0.87119438])

In [ ]:
lcrf[-1].opt_theta